In [1]:
import pandas as pd

In [3]:
answers_csv = 'results-gpt4o-mini.csv'
df = pd.read_csv(answers_csv)
df = df.iloc[:300]
df.head(1)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp


In [11]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Q1

In [16]:
answer_llm0 = df.iloc[0].answer_llm

answer_llm00 = embedding_model.encode(answer_llm0)
answer_llm00

array([-4.22446579e-01, -2.24856094e-01, -3.24058473e-01, -2.84758508e-01,
        7.25637982e-03,  1.01186477e-01,  1.03716679e-01, -1.89983442e-01,
       -2.80597862e-02,  2.71588922e-01, -1.15337484e-01,  1.14666171e-01,
       -8.49585980e-02,  3.32365513e-01,  5.52722663e-02, -2.22195774e-01,
       -1.42540708e-01,  1.02519087e-01, -1.52333796e-01, -2.02912658e-01,
        1.98424123e-02,  8.38147700e-02, -5.68632305e-01,  2.32842825e-02,
       -1.67292669e-01, -2.39256859e-01, -8.05463567e-02,  2.57082582e-02,
       -8.15465227e-02, -7.39290714e-02, -2.61549950e-01,  1.92575473e-02,
        3.22909385e-01,  1.90357044e-01, -9.34726413e-05, -2.13165492e-01,
        2.88941171e-02, -1.79531835e-02, -5.92757724e-02,  1.99918330e-01,
       -4.75172400e-02,  1.71634063e-01, -2.45916881e-02, -9.38061625e-02,
       -3.57002705e-01,  1.33263960e-01,  1.94045797e-01, -1.18530668e-01,
        4.56915230e-01,  1.47728100e-01,  3.35945100e-01, -1.86959475e-01,
        2.45955020e-01, -

# Q2

In [39]:
from tqdm.auto import tqdm

answers_dict = df.to_dict(orient='records')

def cosine_similarity(record):
    answer_llm = record['answer_llm']
    answer_orig = record['answer_orig']
    answer_llm_cos = embedding_model.encode(answer_llm)
    answer_orig_cos = embedding_model.encode(answer_orig)

    return answer_llm_cos.dot(answer_orig_cos)

cos_evaluation_list = []

for record in tqdm(answers_dict):
    cos_sim = cosine_similarity(record)
    cos_evaluation_list.append(cos_sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [40]:
import numpy as np

print("75% percentile", np.percentile(cos_evaluation_list, 75))

75% percentile 31.674312114715576


# Q3

In [41]:
from tqdm.auto import tqdm

answers_dict = df.to_dict(orient='records')

def cosine_similarity_norm(record):
    answer_llm = record['answer_llm']
    answer_orig = record['answer_orig']
    answer_llm_cos = embedding_model.encode(answer_llm)
    answer_orig_cos = embedding_model.encode(answer_orig)

    answer_llm_cos_norm = np.sqrt((answer_llm_cos * answer_llm_cos).sum())
    v_answer_llm_cos_norm = answer_llm_cos / answer_llm_cos_norm

    answer_orig_cos_norm = np.sqrt((answer_orig_cos * answer_orig_cos).sum())
    v_answer_orig_cos_norm = answer_orig_cos / answer_orig_cos_norm

    return v_answer_llm_cos_norm.dot(v_answer_orig_cos_norm)

cos_evaluation_list_normalised = []

for record in tqdm(answers_dict):
    cos_sim_norm = cosine_similarity_norm(record)
    cos_evaluation_list_normalised.append(cos_sim_norm)

  0%|          | 0/300 [00:00<?, ?it/s]

In [42]:
import numpy as np

print("75% percentile on normalised vectors", np.percentile(cos_evaluation_list_normalised, 75))

75% percentile on normalised vectors 0.8362347930669785


# Q4

In [43]:
from rouge import Rouge

In [46]:
rouge_scorer = Rouge()

r = answers_dict[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [47]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

# Q5

In [62]:
f_scores = []

for score in scores.values():
    f_scores.append(score['f'])

print("10th records avg f1 score using Rouge: ",np.average(f_scores))

10th records avg f1 score using Rouge:  0.35490034990035496


# Q6

In [75]:
f1_score_list = []

for record in tqdm(answers_dict):
    scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    f1_score_list.append(scores['rouge-2']['f'])

  0%|          | 0/300 [00:00<?, ?it/s]

In [76]:
df["f1_score"] = f1_score_list
df.iloc[10]

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
f1_score                                                0.216216
Name: 10, dtype: object

In [77]:
df['f1_score'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: f1_score, dtype: float64